# Neurosynth Compose Notebook

After you've built your meta-analysis on [neurosynth compose](https://compose.neurosynth.org), you can use this notebook to execute and view the meta-analytic results.

In [ ]:
#@title Run once to install NiMARE (~1 minute)
%%capture --no-display --no-stderr
%%bash
wget https://raw.githubusercontent.com/neurostuff/neurosynth-compose-notebook/main/requirements.txt
pip install -r requirements.txt

## Run your selected Meta-Analysis

A workflow in NiMARE will interpret the meta-analysis you specified and
run the selected algorithm and corrector (optional) on the studyset selected.

In [ ]:
#@title Insert the Meta-Analysis ID and Execute your Meta-Analysis { vertical-output: true }
from pynsc.run import run  #  special workflow to run meta-analysis
META_ID = "" #@param {type:"string"}
url = meta_result = None
if META_ID is not None:
    url, meta_result = run(META_ID)
else:
    print("UPDATE META_ID VARIABLE TO YOUR META-ANALYSIS")

if url is not None:
  print(f"view the completed meta-analysis here: {url}")

## Create a Report

NiMARE can generate a report of the meta-analysis for you to view and explore. The report provides an overview of the coordinates input into the meta-analysis, key statistical maps, statistically significant clusters, and diagnostics to view the contributions of individual studies to each cluster.


In [ ]:
from nimare.reports import run_reports
run_reports(meta_result, 'report')

## View the Report

You can either download the report using the cell below, or view the contents in google colab two cells below.

In [ ]:
#@title Download the Report
from google.colab import files
import shutil

folder_path = '/content/report'  # Replace with the path to your folder

# Zip the folder
shutil.make_archive('/content/report', 'zip', folder_path)



# Download the zip file
files.download('/content/report.zip')



In [ ]:
#@title Display the Report
import portpicker
import threading
import socket
import IPython

from six.moves import socketserver
from six.moves import SimpleHTTPServer

class V6Server(socketserver.TCPServer):
  address_family = socket.AF_INET6

class Handler(SimpleHTTPServer.SimpleHTTPRequestHandler):
  def end_headers(self):
        self.send_header('Access-Control-Allow-Origin', '*')
        super().end_headers()

port = portpicker.pick_unused_port()

def server_entry():
    httpd = V6Server(('::', port), Handler)
    # Handle a single request then exit the thread.
    httpd.serve_forever()

thread = threading.Thread(target=server_entry)
thread.start()

from google.colab import output
output.serve_kernel_port_as_iframe(port, path='/report/report.html')